In [1]:
from collections import defaultdict, Counter
from google.cloud import bigquery
import pandas as pd
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils
from typing import Dict,List,TypedDict,Optional

2024-04-02 16:07:33.870276: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
nir_t_path = "gs://etldata-prod-search-ranking-data-hkwv8r/data/shared/neural_ir/transformers_hqi_loose/models/2024-03-09/checkpoints/saved_model_04"

In [3]:
nir_t = tf.saved_model.load(nir_t_path)

RuntimeError: Op type not registered 'SentencepieceOp' in binary running on 5943. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib (e.g. `tf.contrib.resampler`), accessing should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.

In [ ]:
nir_t_spec = {
    "listingId": tf.io.FixedLenFeature([], tf.int64),
    **{k:tf.io.FixedLenFeature([], tf.string) for k in ['title', 'tags','taxonomyPath']},
    **{k:tf.io.VarLenFeature(tf.string) for k in ['clickTopQuery', 'cartTopQuery','purchaseTopQuery']}
}
nir_t_schema = schema_utils.schema_from_feature_spec(nir_t_spec)
nir_t_tfex_encoder = tft.coders.ExampleProtoCoder(nir_t_schema)


class NIRTExample(TypedDict):
    listingId: int
    title: str
    tags: str
    taxonomyPath: str

def create_listing_tf_examples(listings: List[NIRTExample]):
    # Returns 1-d string tensor with encoded TF examples
    return tf.constant([nir_t_tfex_encoder.encode(l) for l in listings])

def nir_t_embed_listings(listings: List[NIRTExample], hqi_from:Optional[List[str]]=None):
    # hqi_from: If specified, empty out the HQI fields of a listing and set clickTopQuery to the listing fields in hqi_from
    if hqi_from is not None:
        blank_hqi = {"clickTopQuery": [], "cartTopQuery": [], "purchaseTopQuery": []}
        new_listings = []
        for l in listings:
            new_l = {**l, **blank_hqi}
            new_l['clickTopQuery'] = [l[f] for f in hqi_from]
            new_listings.append(new_l)
        listings = new_listings
    listing_tf_examples = create_listing_tf_examples(listings)
    return nir_t.signatures['embed_listings'](tf_examples=listing_tf_examples)['embedding']

def nir_t_embed_queries(queries: List[str]):
    queries = tf.constant(queries)
    return nir_t.signatures['embed_raw_queries'](queries)['embedding']